In [1]:
import os
from typing import List, Dict
from dotenv import load_dotenv

from langfuse import Langfuse
import pandas as pd
import tiktoken

from config.base_config import rag_config

In [2]:
load_dotenv()

LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY", None)
LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY", None)
LANGFUSE_HOST = "http://localhost:3000"

In [3]:
langfuse = Langfuse(
  secret_key=LANGFUSE_SECRET_KEY,
  public_key=LANGFUSE_PUBLIC_KEY,
  host=LANGFUSE_HOST
)

In [4]:
tokenizer = tiktoken.get_encoding("o200k_base")

In [5]:
pricing = {
    "gpt-4o": {
        "input": 5,
        "output": 15
    },
    "gpt-4o-2024-08-06": {
        "input": 2.5,
        "output": 10
    },
    "gpt-4o-2024-05-13": {
        "input": 5,
        "output": 15
    },
    "gpt-4o-mini": {
        "input": 0.15,
        "output": 0.6
    },
    "gpt-4o-mini-2024-07-18": {
        "input": 0.15,
        "output": 0.6
    },
    "chatgpt-4o-latest": {
        "input": 5.00,
        "output": 15.00
    },
    "gpt-4-turbo": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4-turbo-2024-04-09": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4": {
        "input": 30.00,
        "output": 60.00
    },
    "gpt-4-32k": {
        "input": 60.00,
        "output": 120.00
    },
    "gpt-4-0125-preview": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4-1106-preview": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-4-vision-preview": {
        "input": 10.00,
        "output": 30.00
    },
    "gpt-3.5-turbo-0125": {
        "input": 0.50,
        "output": 1.50
    },
    "gpt-3.5-turbo-instruct": {
        "input": 1.50,
        "output": 2.00
    },
    "gpt-3.5-turbo-1106": {
        "input": 1.00,
        "output": 2.00
    },
    "gpt-3.5-turbo-0613": {
        "input": 1.50,
        "output": 2.00
    },
    "gpt-3.5-turbo-16k-0613": {
        "input": 3.00,
        "output": 4.00
    },
    "gpt-3.5-turbo-0301": {
        "input": 1.50,
        "output": 2.00
    }
 }

In [6]:
model = rag_config["llm"]["model"]

if model in ["gpt-4o", "gpt-4o-2024-05-13", "gpt-4o-2024-08-06", "chatgpt-4o-latest", "gpt-4o-mini", "gpt-4o-mini-2024-07-18"]:
    encoding = "o200k_base"
elif model in ["gpt-4-turbo", "gpt-4-turbo-2024-04-09", "gpt-4-turbo-preview", "gpt-4-0125-preview", "gpt-4-1106-preview", "gpt-4",
               "gpt-4-0613", "gpt-4-0314", "gpt-3.5-turbo-0125", "gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-instruct"]:
    encoding = "cl100k_base"

tokenizer = tiktoken.get_encoding(encoding)

def get_cost(tokenizer, input: List[str], output: List[str], pricing: Dict, model: str):

    n_input_toks = len(tokenizer.encode(input))
    n_output_toks = len(tokenizer.encode(output))
    input_cost = n_input_toks * pricing[model]["input"] / 1_000_000
    output_cost = n_input_toks * pricing[model]["output"] / 1_000_000

    return input_cost + output_cost

### Get traces

In [7]:
traces = langfuse.fetch_traces().data

In [27]:
trace_data = []

for trace in traces:
    
    input = trace.input["args"][1]["query"]
    output = "".join(trace.output) if trace.output else ""
    trace_data.append(
        {
            "id": trace.id,
            "timestamp": trace.timestamp.strftime('%Y-%m-%d %H:%M:%S'),
            "latency": trace.latency,
            "cost": get_cost(tokenizer=tokenizer,
                       input=input,
                       output=output,
                       pricing=pricing,
                       model=model),
            "input": input,
            "output": output
        }
    )

trace_data_df = pd.DataFrame(trace_data)
trace_data_df

,id,timestamp,latency,cost,input,output
0,a7b2eac2-aca2-433b-b674-2df33ba21131,2024-08-22 15:15:34,4.701,0.00016,Wann werden Familienzulagen angemeldet?,Familienzulagen werden in den folgenden Fällen...
1,819230df-d9d4-4baa-8204-1201ae927bd0,2024-08-22 15:15:21,8.393,0.00046,Die Familie lebt in einem EU- oder EFTA-Staat....,Um die Frage systematisch und präzise zu beant...
2,e75ba4be-b61d-4b63-a6c2-71778cb5d618,2024-08-22 15:15:06,7.225,0.00030,Welche Ereignisse müssen vom Familienzulagenbe...,"Um sicherzustellen, dass der Anspruch auf Fami..."
3,053b9c11-55f0-4a49-acf4-f895f2ccfc4b,2024-08-22 15:14:53,7.621,0.00038,Wie werden die Familienzulagen der Familienaus...,"Um die Frage zu beantworten, wie die Familienz..."
4,ae6cc593-8e2f-4772-8b8a-5d8672920733,2024-08-22 15:14:41,6.101,0.00026,Wie können Sie einen bisherigen Anspruch auf A...,Um einen bisherigen Anspruch auf Ausbildungszu...
5,76dbfd74-5d71-482d-a409-788a81c12980,2024-08-22 15:14:30,7.434,0.00058,Wie können Sie Ihren Anspruch auf Familienzula...,Um Ihren Anspruch auf Familienzulagen bei der ...
6,647c024d-5533-4580-9589-70352731fd4e,2024-08-22 15:14:13,10.427,0.00016,Wer hat Anspruch auf Familienzulagen?,Um den Anspruch auf Familienzulagen in der Sch...
7,509bd01c-9ba7-4bda-ba9e-9c4181acfb1d,2024-08-22 15:10:49,0.000,0.00016,Wer hat Anspruch auf Familienzulagen?,
8,48df14f0-50f0-4297-8c74-1b4e209bb5b9,2024-08-22 15:10:32,6.329,0.00016,Wie hoch sind die Familienzulagen?,Die Höhe der Familienzulagen in der Schweiz is...
9,4e9732d1-8c7f-433a-a434-ec36c88cdc2a,2024-08-22 15:10:24,5.032,0.00022,Welche Arten von Familienzulagen werden ausger...,Um die finanzielle Belastung durch Kinder teil...


In [28]:
trace_data_df.cost.sum()

0.008700000000000003

In [29]:
trace_data_df.describe()

,latency,cost
count,23.000000,23.000000
mean,7.277609,0.000378
std,7.682653,0.000200
min,0.000000,0.000160
25%,4.230000,0.000270
50%,6.247000,0.000340
75%,7.900000,0.000440
max,40.415000,0.000900


### Observations

In [30]:
observations = langfuse.fetch_observations(name="retrieve")

In [38]:
obs = {obs.trace_id: obs.output for obs in observations.data}
trace_data_df["retrieval"] = trace_data_df["id"].map(obs)

In [39]:
trace_data_df

,id,timestamp,latency,cost,input,output,retrieval
0,a7b2eac2-aca2-433b-b674-2df33ba21131,2024-08-22 15:15:34,4.701,0.00016,Wann werden Familienzulagen angemeldet?,Familienzulagen werden in den folgenden Fällen...,"[{'id': 112, 'tag': 'Familienzulagen', 'url': ..."
1,819230df-d9d4-4baa-8204-1201ae927bd0,2024-08-22 15:15:21,8.393,0.00046,Die Familie lebt in einem EU- oder EFTA-Staat....,Um die Frage systematisch und präzise zu beant...,"[{'id': 114, 'tag': 'Familienzulagen', 'url': ..."
2,e75ba4be-b61d-4b63-a6c2-71778cb5d618,2024-08-22 15:15:06,7.225,0.00030,Welche Ereignisse müssen vom Familienzulagenbe...,"Um sicherzustellen, dass der Anspruch auf Fami...","[{'id': 109, 'tag': 'Familienzulagen', 'url': ..."
3,053b9c11-55f0-4a49-acf4-f895f2ccfc4b,2024-08-22 15:14:53,7.621,0.00038,Wie werden die Familienzulagen der Familienaus...,"Um die Frage zu beantworten, wie die Familienz...","[{'id': 139, 'tag': 'Familienzulagen', 'url': ..."
4,ae6cc593-8e2f-4772-8b8a-5d8672920733,2024-08-22 15:14:41,6.101,0.00026,Wie können Sie einen bisherigen Anspruch auf A...,Um einen bisherigen Anspruch auf Ausbildungszu...,"[{'id': 120, 'tag': 'Familienzulagen', 'url': ..."
5,76dbfd74-5d71-482d-a409-788a81c12980,2024-08-22 15:14:30,7.434,0.00058,Wie können Sie Ihren Anspruch auf Familienzula...,Um Ihren Anspruch auf Familienzulagen bei der ...,"[{'id': 143, 'tag': None, 'url': 'https://www...."
6,647c024d-5533-4580-9589-70352731fd4e,2024-08-22 15:14:13,10.427,0.00016,Wer hat Anspruch auf Familienzulagen?,Um den Anspruch auf Familienzulagen in der Sch...,"[{'id': 8, 'tag': 'Familienzulagen', 'url': 'h..."
7,509bd01c-9ba7-4bda-ba9e-9c4181acfb1d,2024-08-22 15:10:49,0.000,0.00016,Wer hat Anspruch auf Familienzulagen?,,None
8,48df14f0-50f0-4297-8c74-1b4e209bb5b9,2024-08-22 15:10:32,6.329,0.00016,Wie hoch sind die Familienzulagen?,Die Höhe der Familienzulagen in der Schweiz is...,"[{'id': 8, 'tag': 'Familienzulagen', 'url': 'h..."
9,4e9732d1-8c7f-433a-a434-ec36c88cdc2a,2024-08-22 15:10:24,5.032,0.00022,Welche Arten von Familienzulagen werden ausger...,Um die finanzielle Belastung durch Kinder teil...,"[{'id': 139, 'tag': 'Familienzulagen', 'url': ..."
